In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

import sys

sys.path.insert(0,'../') 
import skseq

from skseq.sequences.sequence import Sequence
from skseq.sequences.sequence_list import SequenceList

from skseq.sequences.label_dictionary import LabelDictionary

import skseq.sequences.structured_perceptron as spc

df_train = pd.read_csv('data/train_data_ner.csv')
df_test = pd.read_csv('data/test_data_ner.csv')
tag_dict = LabelDictionary(label_names=set(df_train.tags))
word_dict = LabelDictionary(label_names=(set(df_train.words) | set(df_test.words)))  # Union of train and test sets of words

train_seq = SequenceList(word_dict, tag_dict)

for _, group in df_train.groupby('sentence_id'):
    train_seq.add_sequence(group.words, group.tags, train_seq.x_dict, train_seq.y_dict)

feature_mapper = skseq.sequences.id_feature.IDFeatures(train_seq)
feature_mapper.build_features()

sp = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper)

/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/otis/Documents/uni/2/NLP/NLP/Assignment_2/skseq/sequences/cython/compute_scores.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /Users/otis/.pyxbld/temp.macosx-10.9-x86_64-cpython-39/pyrex/skseq/sequences/cython/compute_scores.c:698:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1960:
/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warnin

In [3]:
%%prun -s cumulative -T profile_output/fit.txt
sp.fit(feature_mapper.dataset, 1)

Epoch: 0 Accuracy: 0.893947
 
*** Profile printout saved to text file 'profile_output/fit.txt'.


         496331062 function calls in 262.630 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  262.630  262.630 {built-in method builtins.exec}
        1    0.000    0.000  262.630  262.630 <string>:1(<module>)
        1    0.001    0.001  262.630  262.630 structured_perceptron.py:25(fit)
        1    0.129    0.129  262.629  262.629 structured_perceptron.py:58(fit_epoch)
    38366    4.617    0.000  262.500    0.007 structured_perceptron.py:95(perceptron_update)
    38366    0.276    0.000  257.180    0.007 sequence_classifier.py:124(viterbi_decode)
    38366    0.080    0.000  231.091    0.006 discriminative_sequence_classifier.py:27(compute_scores)
    38366   91.706    0.002  231.010    0.006 {skseq.sequences.cython.compute_scores.compute_scores}
231708025  110.695    0.000  128.471    0.000 id_feature.py:113(get_transition_features)
    38366   11.589    0.000   25.605    0.001 sequence_cla

In [4]:
%%prun -s cumulative -T profile_output/compute_scores.txt
np.random.seed(140)
for i in np.random.randint(low=0, high=len(train_seq), size=1000):
  sequence = train_seq[i]
  initial_scores, transition_scores, final_scores, emission_scores = \
    sp.compute_scores(sequence)

 
*** Profile printout saved to text file 'profile_output/compute_scores.txt'.


         12528407 function calls (12528406 primitive calls) in 4.310 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.310    4.310 {built-in method builtins.exec}
        1    0.011    0.011    4.309    4.309 <string>:1(<module>)
     1000    0.001    0.000    4.290    0.004 discriminative_sequence_classifier.py:27(compute_scores)
     1000    1.731    0.002    4.289    0.004 {skseq.sequences.cython.compute_scores.compute_scores}
  6058018    2.033    0.000    2.368    0.000 id_feature.py:113(get_transition_features)
6059020/6059019    0.335    0.000    0.335    0.000 {built-in method builtins.len}
   373354    0.183    0.000    0.183    0.000 id_feature.py:98(get_emission_features)
        1    0.004    0.004    0.005    0.005 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
    17000    0.003    0.000    0.003    0.000 id_feature.py:124(get_initial_features)
    17000    0.

In [5]:
%%timeit
sequence = train_seq[np.random.randint(low=0,high=len(train_seq))]
initial_scores, transition_scores, final_scores, emission_scores = \
  sp.compute_scores(sequence)

2.83 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%prun -s cumulative -T profile_output/run_viterbi.txt
for i in range(1000):
  best_states, total_score = sp.decoder.run_viterbi(initial_scores,
                                                    transition_scores,
                                                    final_scores,
                                                    emission_scores)

 
*** Profile printout saved to text file 'profile_output/run_viterbi.txt'.


         336003 function calls in 0.503 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.503    0.503 {built-in method builtins.exec}
        1    0.002    0.002    0.503    0.503 <string>:1(<module>)
     1000    0.235    0.000    0.501    0.001 sequence_classification_decoder.py:84(run_viterbi)
    46000    0.037    0.000    0.218    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    21000    0.015    0.000    0.160    0.000 <__array_function__ internals>:177(amax)
    21000    0.017    0.000    0.128    0.000 fromnumeric.py:2675(amax)
    21000    0.029    0.000    0.111    0.000 fromnumeric.py:69(_wrapreduction)
    21000    0.016    0.000    0.089    0.000 <__array_function__ internals>:177(argmax)
    21000    0.071    0.000    0.071    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    21000    0.016    0.000    0.052    0.000 fromnumeric.py:1127(argm

In [7]:
%%timeit
sp.decoder.run_viterbi(initial_scores,
                       transition_scores,
                       final_scores,
                       emission_scores)

318 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
a1, a2 = sp.decoder.run_viterbi(initial_scores,
                       transition_scores,
                       final_scores,
                       emission_scores)

In [15]:
b1, b2 = sp.decoder.run_viterbi(initial_scores,
                       transition_scores,
                       final_scores,
                       emission_scores)

In [21]:
(a1 == b1).all()

True

In [22]:
b2 == a2

True

In [7]:
%load_ext cython

In [21]:
%%cython -a -c=-DCYTHON_TRACE=1
import numpy as np
cimport numpy as np
cimport cython
np.import_array()

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def compute_scores(obj, sequence):
    cdef int num_states = obj.get_num_states()
    cdef int length = len(sequence.x)
    cdef double[:,:] emission_scores = np.zeros([length, num_states])
    cdef double[:] initial_scores = np.zeros(num_states)
    cdef double[:,:,:] transition_scores = np.zeros([length-1, num_states, num_states])
    cdef double[:] final_scores = np.zeros(num_states)

    cdef float score
    cdef int feat_id, pos, tag_id, prev_tag_id
    cdef np.ndarray[double, ndim=1] parameters = obj.parameters
    # cdef np.ndarray[np.int, ndim=1] initial_features, emission_features, transition_features

    # Initial position.
    for tag_id in range(num_states):
        feat_id = obj.feature_mapper.get_initial_feature(sequence, tag_id)
        score = parameters[feat_id] if feat_id != -1 else 0.0
        initial_scores[tag_id] = score


    # Intermediate position.
    for pos in range(length):
        for tag_id in range(num_states):
            feat_ids = obj.feature_mapper.get_emission_features(sequence, pos, tag_id)
            score = 0.0
            for feat_id in feat_ids:
                score += parameters[feat_id]
            emission_scores[pos, tag_id] = score
        if pos > 0:
            for tag_id in range(num_states):
                for prev_tag_id in range(num_states):
                    feat_id = obj.feature_mapper.get_transition_feature(
                        sequence, pos, tag_id, prev_tag_id)
                    score = parameters[feat_id]  if feat_id != -1 else 0.0
                    transition_scores[pos-1, tag_id, prev_tag_id] = score

    # Final position.
    for prev_tag_id in range(num_states):
        feat_id = obj.feature_mapper.get_final_feature(sequence, prev_tag_id)
        score = parameters[feat_id] if feat_id != -1 else 0.0
        final_scores[prev_tag_id] = score


    return initial_scores, transition_scores, final_scores, emission_scores

In file included from /Users/otis/.ipython/cython/_cython_magic_322bc0b7aa3ac7d001a12583731d463b.c:728:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1960:
/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.
ld: warning: -pie being ignored. It is only used when linking a main executable


In [24]:
%%timeit
sequence = train_seq[np.random.randint(low=0,high=len(train_seq))]
initial_scores, transition_scores, final_scores, emission_scores = \
  compute_scores(sp, sequence)

3.11 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
%%prun -s cumulative -T profile_output/compute_scores.txt
np.random.seed(140)
for i in np.random.randint(low=0, high=len(train_seq), size=1000):
  sequence = train_seq[i]
  initial_scores, transition_scores, final_scores, emission_scores = \
    compute_scores(sp, sequence)

 
*** Profile printout saved to text file 'profile_output/compute_scores.txt'.
         12527407 function calls (12527406 primitive calls) in 4.656 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.656    4.656 {built-in method builtins.exec}
        1    0.012    0.012    4.656    4.656 <string>:1(<module>)
     1000    1.652    0.002    4.638    0.005 {_cython_magic_6701dbb792c705a50fbc429fee20d4ad.compute_scores}
  6058018    2.408    0.000    2.758    0.000 id_feature.py:113(get_transition_features)
6059020/6059019    0.350    0.000    0.350    0.000 {built-in method builtins.len}
   373354    0.220    0.000    0.220    0.000 id_feature.py:98(get_emission_features)
     1000    0.006    0.000    0.006    0.000 sequence_list.py:41(__getitem__)
    17000    0.004    0.000    0.004    0.000 id_feature.py:124(get_initial_features)
    17000    0.003    0.000    0.003    0.000 id_feature.py:1

In [7]:
sequence = train_seq[5]

In [12]:
obj = sp
num_states = obj.get_num_states()
length = len(sequence.x)
emission_scores = np.zeros([length, num_states])
initial_scores = np.zeros(num_states)
transition_scores = np.zeros([length-1, num_states, num_states])
final_scores = np.zeros(num_states)

for pos in range(length):
    if pos > 0:
        for tag_id in range(num_states):
            for prev_tag_id in range(num_states):
                feat_id = obj.feature_mapper.get_transition_feature(
                    sequence, pos-1, tag_id, prev_tag_id)


In [28]:
%%timeit
for pos in range(length):
    if pos > 0:
        for tag_id in range(num_states):
            for prev_tag_id in range(num_states):
                feat_id = obj.feature_mapper.get_transition_feature2(
                    sequence, pos-1, tag_id, prev_tag_id)

3.63 ms ± 298 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
for pos in range(length):
    if pos > 0:
        for tag_id in range(num_states):
            for prev_tag_id in range(num_states):
                feat_id = obj.feature_mapper.get_transition_feature(
                    sequence, pos-1, tag_id, prev_tag_id)

6.45 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%%prun -s cumulative -T test2.txt
for _ in range(1000):
    for pos in range(length):
        if pos > 0:
            for tag_id in range(num_states):
                for prev_tag_id in range(num_states):
                    feat_id = obj.feature_mapper.get_transition_feature2(
                        sequence, pos-1, tag_id, prev_tag_id)

 
*** Profile printout saved to text file 'test2.txt'.
         12138003 function calls in 5.831 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.831    5.831 {built-in method builtins.exec}
        1    2.464    2.464    5.831    5.831 <string>:1(<module>)
  6069000    2.919    0.000    3.367    0.000 id_feature.py:119(get_transition_feature2)
  6069000    0.448    0.000    0.448    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [24]:
%%prun -s cumulative -T test.txt
for _ in range(1000):
    for pos in range(length):
        if pos > 0:
            for tag_id in range(num_states):
                for prev_tag_id in range(num_states):
                    feat_id = obj.feature_mapper.get_transition_feature(
                        sequence, pos-1, tag_id, prev_tag_id)

 
*** Profile printout saved to text file 'test.txt'.
         12138003 function calls in 8.024 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    8.024    8.024 {built-in method builtins.exec}
        1    2.581    2.581    8.024    8.024 <string>:1(<module>)
  6069000    5.006    0.000    5.443    0.000 id_feature.py:111(get_transition_feature)
  6069000    0.437    0.000    0.437    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}